In [ ]:
!pip install git+https://github.com/UKPLab/sentence-transformers.git
from huggingface_hub import notebook_login
notebook_login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/UKPLab/sentence-transformers.git to /tmp/pip-req-build-wnwz1feg
  Running command git clone --filter=blob:none --quiet https://github.com/UKPLab/sentence-transformers.git /tmp/pip-req-build-wnwz1feg
  Resolved https://github.com/UKPLab/sentence-transformers.git to commit feafe1e842603f8dabe511f2137942e666f25833
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126128 sha256=6c3275b604e857baa9be1aa50bb70a3ae22230f6a1c268aede84878bd6115d19
  Stored in directo

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import random
from google.colab import drive
drive.mount('/content/gdrive')

# Run locally
#DATA_PATH = ''

# KW
#DATA_PATH = '/content/gdrive/MyDrive/CSI6900/'

# FZ
DATA_PATH = 'gdrive/MyDrive/CSI6900/'

device = torch.device("mps" if getattr(torch,'has_mps',False) else "cuda:0" if torch.cuda.is_available() else "cpu")
device

Mounted at /content/gdrive


device(type='cuda', index=0)

## Load data

In [ ]:
infoSheet = pd.read_csv(DATA_PATH + "data/infoSheets_2023-05-18.csv")
print(infoSheet.isnull().sum())
infoSheet.dropna(subset=['abstract_en'], inplace=True)
infoSheet.reset_index(drop=True, inplace=True)
print('\nNumber of rows: ', len(infoSheet.index))
infoSheet.head(5)

ID                        0
name_en                   0
name_fr                 138
abstract_en              20
abstract_fr             146
description_en            0
description_fr          140
taxonomy heading ids      0
dtype: int64

Number of rows:  326


,ID,name_en,name_fr,abstract_en,abstract_fr,description_en,description_fr,taxonomy heading ids
0,84606,ADHD Medication Side Effects: Low Appetite and...,NaN,Stimulants prescribed for ADHD can lead to red...,NaN,Background\r\nStimulant medications for attent...,NaN,0
1,92619,5-HTP (5-hydroxytryptophan),NaN,5-HTP (5-Hydroxytryptophan) is a natural subst...,NaN,What is 5-HTP?\r\n5-HTP (5-Hydroxytryptophan) ...,NaN,0
2,50150,A Simple Way to Swallow Pills: The Head Postur...,Truc simple pour avaler les pilules: La techni...,"Swallowing pills can hard for many children, y...","Il n’est pas seul! Beaucoup d’enfants, de jeun...",\r\n\t\r\n\t\tDoes your child or teen have pro...,\r\n\t\r\n\t\tVotre enfant a-t-il de la diffic...,0
3,8920,Abuse and Domestic Violence,Maltraitance et violence familiale,"Abuse is behaviour used to intimidate, isolate...",La maltraitance est un comportement visant à i...,\r\n\tWhat is Abuse and Domestic Violence?\r\n...,\r\n\tQu&#39;est-ce que la maltraitance et la ...,"0,219,58,8,7,6,50,9,36,54,37"
4,69660,"ADHD in Children, Youth and Adults: Informatio...",NaN,Attention deficit hyperactivity disorder (ADHD...,NaN,"\r\n\tAbbreviations\r\n\r\n\tADHD, attention-d...",NaN,"0,13"


In [ ]:
records = pd.read_json(DATA_PATH + 'data/records.json')
records = records.drop(['created_at', 'updated_at', 'deleted_at', 'publish', 'academic_credentials', 'age_max', 'age_min', 'last_name',
                        'latitude', 'longitude', 'name_of_private_practice', 'fee_description',	'fee_type',	'first_name',	'languages',
                        'organization_type', 'original_id',	'record_type',	'salutation_type', 'website'], axis=1)

taxonomy = pd.read_json(DATA_PATH + 'data/taxonomy_headings.json')
taxonomy = taxonomy.drop(['created_at',	'updated_at',	'deleted_at', 'alias_of_id', 'short_description',	'original_id'], axis=1)

taxonomy.head(10)

,id,name,description,translations
0,1,Root,Root,"{""name"":{""en"":""Root"",""fr"":null},""description"":..."
1,2,All Mental Health Resources,<p>\r\n\tThe listings of mental health resourc...,"{""name"":{""en"":""All Mental Health Resources"",""f..."
2,3,Crisis and Emergency,<p>\r\n\tRefers to all programs that provide i...,"{""name"":{""en"":""Crisis and Emergency"",""fr"":""Res..."
3,4,"System Navigation, including Information and R...","<p>\r\n\tAre you looking for help, but don&#39...","{""name"":{""en"":""System Navigation, including In..."
4,5,Child Welfare including Children's Aid Society...,<p>The child welfare / child protection system...,"{""name"":{""en"":""Child Welfare including Childre..."
5,6,Emergency Shelter and Housing,<p>\r\n\tThere are various shelters that peopl...,"{""name"":{""en"":""Emergency Shelter and Housing"",..."
6,7,Hospital Emergency Department,<p>\r\n\tIs there an emergency such as medical...,"{""name"":{""en"":""Hospital Emergency Department"",..."
7,8,"Crisis Lines including Telephone, Online and Chat",<p>\r\n\tAre you in a crisis? Crisis lines off...,"{""name"":{""en"":""Crisis Lines including Telephon..."
8,9,Psychiatrists,<p>\r\n\tPsychiatrists are medical doctors who...,"{""name"":{""en"":""Psychiatrists"",""fr"":""Psychiatre..."
9,10,A-Z Mental Health Conditions and Topics,<p>\r\n\tAlphabetical list of mental health to...,"{""name"":{""en"":""A-Z Mental Health Conditions an..."


## Pre-processing

In [ ]:
!cp /content/gdrive/MyDrive/CSI6900/pre_processing.py /content

In [ ]:
from pre_processing import remove_empty, remove_HTML, remove_new_line

infoSheet['description_en'] = infoSheet['description_en'].apply(lambda x: remove_HTML(x))
infoSheet['description_en'] = infoSheet['description_en'].apply(lambda x: remove_new_line(x))

print('Length of taxonomy before preprocessing:', len(taxonomy.index))
taxonomy['description'] = taxonomy['description'].apply(lambda x: remove_HTML(x))
taxonomy['description'] = taxonomy['description'].apply(lambda x: remove_new_line(x))
# taxonomy = remove_empty('description', taxonomy)
print('Length of taxonomy after preprocessing:', len(taxonomy.index))

print('Length of records before preprocessing:', len(records.index))
records['description'] = records['description'].apply(lambda x: remove_HTML(x))
records['description'] = records['description'].apply(lambda x: remove_new_line(x))
records = remove_empty('description', records)
print('Length of records after preprocessing:', len(records.index))

Length of taxonomy before preprocessing: 277
Length of taxonomy after preprocessing: 277
Length of records before preprocessing: 6406
Length of records after preprocessing: 6239


## SGPT

### Prediction

In [ ]:
model = SentenceTransformer("Muennighoff/SGPT-1.3B-weightedmean-nli-bitfit")

In [ ]:
taxonomy_description = taxonomy['description'].tolist()
taxonomy_name = taxonomy['name'].tolist()

taxonomy_data = []
for i in range(len(taxonomy_name)):
    taxonomy_data.append(taxonomy_name[i] + ': ' + taxonomy_description[i])

taxonomy_embeddings = model.encode(taxonomy_data, convert_to_tensor=True).to(device)

In [ ]:
# Randomly choose 10 infoSheets
search_term_indices = [random.randint(0, len(infoSheet.index)) for i in range(10)]
search_term_indices

[191, 216, 49, 98, 57, 86, 157, 195, 122, 162]

In [ ]:
infoSheet_abstract = infoSheet['description_en'].tolist()
infoSheet_name = infoSheet['name_en'].tolist()

# Concat name and abstraction
infoSheet_data = []
for i in range(len(infoSheet_name)):
    if i in search_term_indices:
        infoSheet_data.append(infoSheet_name[i] + ': ' + infoSheet_abstract[i])

infoSheet_embeddings = model.encode(infoSheet_data, convert_to_tensor=True).to(device)

In [ ]:
hits = util.semantic_search(query_embeddings=infoSheet_embeddings, corpus_embeddings=taxonomy_embeddings)

In [ ]:
predictions = {'infoSheet_id': [], 'pred_taxonomy_id': [], 'score': [], 'gold_taxonomy_id': []}
for idx in range(len(hits)):
    for each in hits[idx]:
        if each['score'] >= 0.7:
            predictions['infoSheet_id'].append(search_term_indices[idx])
            predictions['pred_taxonomy_id'].append(each['corpus_id'])
            predictions['score'].append(each['score'])
            predictions['gold_taxonomy_id'].append(infoSheet['taxonomy heading ids'][search_term_indices[idx]].split(','))

predictions = pd.DataFrame(predictions)
predictions.head(10)

,infoSheet_id,pred_taxonomy_id,score,gold_taxonomy_id
0,191,75,0.744044,[0]
1,191,106,0.716783,[0]
2,98,216,0.881918,"[0, 217]"
3,98,76,0.721461,"[0, 217]"
4,57,26,0.736842,"[0, 22]"
5,86,275,0.938887,"[0, 79, 50, 9, 36]"


### Evaluation

In [ ]:
infoSheet.loc[infoSheet.index == 98]

,ID,name_en,name_fr,abstract_en,abstract_fr,description_en,description_fr,taxonomy heading ids
98,80568,Dialectical Behaviour Therapy (DBT) for Psychi...,La thérapie comportementale dialectique (TCD) ...,Dialectical behavioral therapy (DBT) is an evi...,La thérapie comportementale dialectique (TCD) ...,What is DBT? Dialectical Behaviour Therapy (DB...,Qu'est-ce que la thérapie comportementale dial...,"0,217"


In [ ]:
infoSheet['abstract_en'][98]

'Dialectical behavioral therapy (DBT) is an evidence-based treatment that is helpful for borderline personality disorder and a wide range of conditions. It helps with self-injury and suicidal behaviours, as well as reducing rates of hospitalization. DBT teaches people how to handle intense emotions by learning specific skills such as mindfulness, interpersonal effectiveness, distress tolerance and emotional regulation.'

In [ ]:
infoSheet['name_en'][98]

'Dialectical Behaviour Therapy (DBT) for Psychiatry Residents'

In [ ]:
taxonomy.loc[taxonomy.index == 216]

,id,name,description,translations
216,217,Dialectical Behaviour Therapy (DBT),Dialectical behavior therapy (DBT) is a compre...,"{""name"":{""en"":""Dialectical Behaviour Therapy (..."


In [ ]:
taxonomy.loc[taxonomy.index == 217]

,id,name,description,translations
217,218,Emotionally Focused Therapy (EFT),"EFT is a short term (8-20 sessions), structure...","{""name"":{""en"":""Emotionally Focused Therapy (EF..."


In [ ]:
aretrieved_relevant = 0

for idx, row in predictions.iterrows():
    if str(row['pred_taxonomy_id']) in row['gold_taxonomy_id']:
        retrieved_relevant += 1

print(retrieved_relevant)
print('Precision: ' + str(retrieved_relevant / len(predictions.index)))

0
Precision: 0.0


## RoBERTa

### Prediction

In [ ]:
model = SentenceTransformer("AIMH/mental-roberta-large")

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/AIMH_mental-roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/AIMH_mental-roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this mode

In [ ]:
taxonomy_description = taxonomy['description'].tolist()
taxonomy_name = taxonomy['name'].tolist()

taxonomy_data = []
for i in range(len(taxonomy_name)):
    taxonomy_data.append(taxonomy_name[i] + ': ' + taxonomy_description[i])

taxonomy_embeddings = model.encode(taxonomy_data, convert_to_tensor=True).to(device)

In [ ]:
# Randomly choose 10 infoSheets
search_term_indices = [random.randint(0, len(infoSheet.index)) for i in range(10)]
search_term_indices

[158, 276, 213, 65, 175, 94, 238, 197, 169, 304]

In [ ]:
infoSheet_abstract = infoSheet['description_en'].tolist()
infoSheet_name = infoSheet['name_en'].tolist()

# Concat name and abstraction
infoSheet_data = []
for i in range(len(infoSheet_name)):
    if i in search_term_indices:
        infoSheet_data.append(infoSheet_name[i] + ': ' + infoSheet_abstract[i])

infoSheet_embeddings = model.encode(infoSheet_data, convert_to_tensor=True).to(device)

In [ ]:
hits = util.semantic_search(query_embeddings=infoSheet_embeddings, corpus_embeddings=taxonomy_embeddings)

In [ ]:
predictions = {'infoSheet_id': [], 'pred_taxonomy_id': [], 'score': [], 'gold_taxonomy_id': []}
for idx in range(len(hits)):
    for each in hits[idx]:
        if each['score'] >= 0.7:
            predictions['infoSheet_id'].append(search_term_indices[idx])
            predictions['pred_taxonomy_id'].append(each['corpus_id'])
            predictions['score'].append(each['score'])
            predictions['gold_taxonomy_id'].append(infoSheet['taxonomy heading ids'][search_term_indices[idx]].split(','))

predictions = pd.DataFrame(predictions)
predictions.head(150)

,infoSheet_id,pred_taxonomy_id,score,gold_taxonomy_id
0,158,8,0.955985,"[0, 219, 58]"
1,158,97,0.955768,"[0, 219, 58]"
2,158,90,0.955034,"[0, 219, 58]"
3,158,240,0.953157,"[0, 219, 58]"
4,158,230,0.952747,"[0, 219, 58]"
...,...,...,...,...
95,304,262,0.956016,"[0, 67]"
96,304,21,0.954964,"[0, 67]"
97,304,69,0.954514,"[0, 67]"
98,304,274,0.951987,"[0, 67]"


In [ ]:
result_df = pd.DataFrame(columns=['infoSheet_index', 'infoSheet_name', 'infoSheet_abstract', 'taxonomy_heading_ids', 'taxonomy_index', 'taxonomy_name', 'score', 'check'])
result_df['infoSheet_index']= predictions['infoSheet_id']
result_df['taxonomy_heading_ids'] = predictions['gold_taxonomy_id']
result_df['taxonomy_index'] = predictions['pred_taxonomy_id']
result_df['score'] = predictions['score']

for idx,row in predictions.iterrows():
    result_df['infoSheet_name'][idx] = infoSheet['name_en'][row[0]]
    result_df['infoSheet_abstract'][idx] = infoSheet['abstract_en'][row[0]]
    result_df['taxonomy_name'][idx] = taxonomy['name'][row[1]]

result_df.to_csv('mental_RoBERTa_infosheet.csv', index=False)

<ipython-input-46-2e9e155f2eb6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['infoSheet_name'][idx] = infoSheet['name_en'][row[0]]
<ipython-input-46-2e9e155f2eb6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['infoSheet_abstract'][idx] = infoSheet['abstract_en'][row[0]]
<ipython-input-46-2e9e155f2eb6>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['taxonomy_name'][idx] = taxonomy['name'][row[1]]


### Evaluate

In [ ]:
retrieved_relevant = 0

for idx, row in predictions.iterrows():
    if str(row['pred_taxonomy_id']) in row['gold_taxonomy_id']:
        retrieved_relevant += 1

print(retrieved_relevant)
print('Precision: ' + str(retrieved_relevant / len(predictions.index)))

2
Precision: 0.02


## XLNet

### Prediction

In [ ]:
model = SentenceTransformer("AIMH/mental-xlnet-base-cased")

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/AIMH_mental-xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
taxonomy_description = taxonomy['description'].tolist()
taxonomy_name = taxonomy['name'].tolist()

taxonomy_data = []
for i in range(len(taxonomy_name)):
    taxonomy_data.append(taxonomy_name[i] + ': ' + taxonomy_description[i])

taxonomy_embeddings = model.encode(taxonomy_data, convert_to_tensor=True).to(device)

OverflowError: ignored

In [ ]:
# Randomly choose 10 infoSheets
search_term_indices = [random.randint(0, len(infoSheet.index)) for i in range(10)]
search_term_indices

In [ ]:
infoSheet_abstract = infoSheet['description_en'].tolist()
infoSheet_name = infoSheet['name_en'].tolist()

# Concat name and abstraction
infoSheet_data = []
for i in range(len(infoSheet_name)):
    if i in search_term_indices:
        infoSheet_data.append(infoSheet_name[i] + ': ' + infoSheet_abstract[i])

infoSheet_embeddings = model.encode(infoSheet_data, convert_to_tensor=True).to(device)

In [ ]:
hits = util.semantic_search(query_embeddings=infoSheet_embeddings, corpus_embeddings=taxonomy_embeddings)

In [ ]:
predictions = {'infoSheet_id': [], 'pred_taxonomy_id': [], 'score': [], 'gold_taxonomy_id': []}
for idx in range(len(hits)):
    for each in hits[idx]:
        if each['score'] >= 0.7:
            predictions['infoSheet_id'].append(search_term_indices[idx])
            predictions['pred_taxonomy_id'].append(each['corpus_id'])
            predictions['score'].append(each['score'])
            predictions['gold_taxonomy_id'].append(infoSheet['taxonomy heading ids'][search_term_indices[idx]].split(','))

predictions = pd.DataFrame(predictions)
predictions.head(10)

### Evaluata

In [ ]:
retrieved_relevant = 0

for idx, row in predictions.iterrows():
    if str(row['pred_taxonomy_id']) in row['gold_taxonomy_id']:
        retrieved_relevant += 1

print(retrieved_relevant)
print('Precision: ' + str(retrieved_relevant / len(predictions.index)))

## LLaMA

### Prediction

## T5

### Prediction

In [ ]:
model = SentenceTransformer("t5-base")

In [ ]:
taxonomy_description = taxonomy['description'].tolist()
taxonomy_name = taxonomy['name'].tolist()

taxonomy_data = []
for i in range(len(taxonomy_name)):
    taxonomy_data.append(taxonomy_name[i] + ': ' + taxonomy_description[i])

taxonomy_embeddings = model.encode(taxonomy_data, convert_to_tensor=True).to(device)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Randomly choose 10 infoSheets
search_term_indices = [random.randint(0, len(infoSheet.index)) for i in range(10)]
search_term_indices

[81, 92, 149, 129, 115, 310, 239, 146, 119, 298]

In [ ]:
infoSheet_abstract = infoSheet['description_en'].tolist()
infoSheet_name = infoSheet['name_en'].tolist()

# Concat name and abstraction
infoSheet_data = []
for i in range(len(infoSheet_name)):
    if i in search_term_indices:
        infoSheet_data.append(infoSheet_name[i] + ': ' + infoSheet_abstract[i])

infoSheet_embeddings = model.encode(infoSheet_data, convert_to_tensor=True).to(device)

OutOfMemoryError: ignored

In [ ]:
hits = util.semantic_search(query_embeddings=infoSheet_embeddings, corpus_embeddings=taxonomy_embeddings)

In [ ]:
predictions = {'infoSheet_id': [], 'pred_taxonomy_id': [], 'score': [], 'gold_taxonomy_id': []}
for idx in range(len(hits)):
    for each in hits[idx]:
        if each['score'] >= 0.7:
            predictions['infoSheet_id'].append(search_term_indices[idx])
            predictions['pred_taxonomy_id'].append(each['corpus_id'])
            predictions['score'].append(each['score'])
            predictions['gold_taxonomy_id'].append(infoSheet['taxonomy heading ids'][search_term_indices[idx]].split(','))

predictions = pd.DataFrame(predictions)
predictions.head(10)

### Evaluate

In [ ]:
retrieved_relevant = 0

for idx, row in predictions.iterrows():
    if str(row['pred_taxonomy_id']) in row['gold_taxonomy_id']:
        retrieved_relevant += 1

print(retrieved_relevant)
print('Precision: ' + str(retrieved_relevant / len(predictions.index)))